# 04 - Full waveform inversion with Devito and Dask

## Introduction

In this tutorial we show how [Devito](http://www.opesci.org/devito-public) and [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) are used with [Dask](https://dask.pydata.org/en/latest/#dask) to perform [full waveform inversion](https://www.slim.eos.ubc.ca/research/inversion) (FWI) on distributed memory parallel computers.

## scipy.optimize.minimize 

In this tutorial we use [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) to solve the FWI gradient based minimization problem rather than the simple grdient decent algorithm in the previous tutorial.

```python
scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
```

> Minimization of scalar function of one or more variables.
>
> In general, the optimization problems are of the form:
>
> minimize f(x) subject to
>
> g_i(x) >= 0,  i = 1,...,m
> h_j(x)  = 0,  j = 1,...,p
> where x is a vector of one or more variables. g_i(x) are the inequality constraints. h_j(x) are the equality constrains.

[scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) provides a wide variety of methods for solving minimization problems depending on the context. Here we are going to focus on using L-BFGS via [scipy.optimize.minimize(method=’L-BFGS-B’)](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html#optimize-minimize-lbfgsb)

```python
scipy.optimize.minimize(fun, x0, args=(), method='L-BFGS-B', jac=None, bounds=None, tol=None, callback=None, options={'disp': None, 'maxls': 20, 'iprint': -1, 'gtol': 1e-05, 'eps': 1e-08, 'maxiter': 15000, 'ftol': 2.220446049250313e-09, 'maxcor': 10, 'maxfun': 15000})```

The argument `fun` is a callable function that returns the misfit between the simulated and the observed data. If `jac` is a Boolean and is `True`, `fun` is assumed to return the gradient along with the objective function - as is our case when applying the adjoint-state method.

## What is Dask?

> [Dask](https://dask.pydata.org/en/latest/#dask) is a flexible parallel computing library for analytic computing.
>
> Dask is composed of two components:
>
> * Dynamic task scheduling optimized for computation...
> * “Big Data” collections like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of the dynamic task schedulers.
>
> Dask emphasizes the following virtues:
> 
> * Familiar: Provides parallelized NumPy array and Pandas DataFrame objects
> * Flexible: Provides a task scheduling interface for more custom workloads and integration with other projects.
> * Native: Enables distributed computing in Pure Python with access to the PyData stack.
> * Fast: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
> * Scales up: Runs resiliently on clusters with 1000s of cores
> * Scales down: Trivial to set up and run on a laptop in a single process
> * Responsive: Designed with interactive computing in mind it provides rapid feedback and diagnostics to aid humans

**We are going to use it here to parallelise the computation of the functional and gradient as this is the vast bulk of the computational expense of FWI and it is trivially parallel over data shots.**

## Setting up (synthetic) data
In a real world scenario we work with collected seismic data; for the tutorial we know what the actual solution is and we are using the workers to also generate the synthetic data.

In [1]:
#NBVAL_IGNORE_OUTPUT

# Set up inversion parameters.
param = {'t0': 0.,
         'tn': 1000.,              # Simulation last 1 second (1000 ms)
         'f0': 0.010,              # Source peak frequency is 10Hz (0.010 kHz)
         'nshots': 5,              # Number of shots to create gradient from
         'm_bounds': (0.08, 0.25), # Set the min and max slowness
         'shape': (101, 101),      # Number of grid points (nx, nz).
         'spacing': (10., 10.),    # Grid spacing in m. The domain size is now 1km by 1km.
         'origin': (0, 0),         # Need origin to define relative source and receiver locations.
         'nbpml': 40}              # nbpml thickness.

import numpy as np

import scipy
from scipy import signal, optimize

from devito import Grid

from distributed import Client, LocalCluster, wait

import cloudpickle as pickle

# Import acoustic solver, source and receiver modules.
from examples.seismic import Model, demo_model
from examples.seismic.acoustic import AcousticWaveSolver
from examples.seismic import TimeAxis, PointSource, RickerSource, Receiver

# Import convenience function for plotting results
from examples.seismic import plot_image

def get_true_model():
    ''' Define the test phantom; in this case we are using
    a simple circle so we can easily see what is going on.
    '''
    return demo_model('circle-isotropic', vp=3.0, vp_background=2.5, 
                      origin=param['origin'], shape=param['shape'],
                      spacing=param['spacing'], nbpml=param['nbpml'])

def get_initial_model():
    '''The initial guess for the subsurface model.
    '''
    return demo_model('circle-isotropic', vp=2.5, vp_background=2.5, 
                      origin=param['origin'], shape=param['shape'],
                      spacing=param['spacing'], nbpml=param['nbpml'])

def wrap_model(x, astype=None):
    '''Wrap a flat array as a subsurface model.
    '''
    model = get_initial_model()
    if astype:
        model.m.data[:] = x.astype(astype).reshape(model.m.data.shape)
    else:
        model.m.data[:] = x.reshape(model.m.data.shape)
    return model

def get_current_model(param):
    """ Returns the current model. This is used by the
    worker to get the current model.
    """
    model = wrap_model(np.load(param['model']))
    return model

def dump_model(param, model):
    ''' Dump model to disk.
    '''
    np.save(param['model'], model.astype(np.float32))
    
def load_shot_data(shot_id, dt):
    ''' Load shot data from disk, resampling to the model time step.
    '''
    pkl = pickle.load(open("shot_%d.p"%shot_id, "rb"))
    pkl_grid = Grid(extent=pkl['grid_extent'],
                    shape=pkl['grid_shape'],
                    dimensions=pkl['grid_dimensions'])
    time_range = pkl['time_range']
    src = PointSource(name=pkl['src.name'], grid=pkl_grid,
                      npoint=pkl['src.coordinates'].shape[0],
                      time_range=time_range, data=pkl['src.data'],
                      coordinates=pkl['src.coordinates'])
    rec = PointSource(name=pkl['rec.name'], grid=pkl_grid,
                      time_range=time_range, data=pkl['rec.data'],
                      npoint=pkl['rec.coordinates'].shape[0])
    rec.coordinates.data[:] = pkl['rec.coordinates']
    
    return src.resample(dt), rec.resample(dt)

def dump_shot_data(shot_id, src, rec):
    ''' Dump shot data to disk.
    '''
    pickle.dump({'grid_extent':src.grid.extent,
                 'grid_shape':src.grid.shape,
                 'grid_dimensions':src.grid.dimensions,
                 'time_range':src.time_range,
                 'src.name':src.name,
                 'src.data':np.array(src.data),
                 'src.coordinates':np.array(src.coordinates.data),
                 'rec.name':rec.name,
                 'rec.data':np.array(rec.data),
                 'rec.coordinates':np.array(rec.coordinates.data)},
                open('shot_%d.p'%shot_id, "wb"))
    
def generate_shotdata_i(param):
    """ Inversion crime alert! Here the worker is creating the
        'observed' data using the real model. For a real case
        the worker would be reading seismic data from disk.
    """
    true_model = get_true_model()
    shot_id = param['shot_id']
    
    # Time step from model grid spacing
    dt = true_model.critical_dt

    # Set up source data and geometry.

    time_range = TimeAxis(start=param['t0'], stop=param['tn'], step=dt)
    src = RickerSource(name='src', grid=true_model.grid, f0=param['f0'],
                       time_range=time_range)

    src.coordinates.data[0, :] = [30, param['shot_id']*1000./(param['nshots']-1)]
    
    # Number of receiver locations per shot.
    nreceivers = 101

    # Set up receiver data and geometry.
    rec = Receiver(name='rec', grid=true_model.grid, time_range=time_range,
                   npoint=nreceivers)
    rec.coordinates.data[:, 1] = np.linspace(0, true_model.domain_size[0], num=nreceivers)
    rec.coordinates.data[:, 0] = 980. # 20m from the right end

    # Set up solver.
    solver = AcousticWaveSolver(true_model, src, rec, space_order=4)

    # Generate synthetic receiver data from true model.
    true_d, _, _ = solver.forward(src=src, m=true_model.m)

    dump_shot_data(shot_id, src, true_d)

def generate_shotdata(param):
    # Define work list
    work = [dict(param) for i in range(param['nshots'])]
    for i in  range(param['nshots']):
        work[i]['shot_id'] = i
        generate_shotdata_i(work[i])
        
    # Map worklist to cluster
    client = Client(cluster)

    futures = client.map(generate_shotdata_i, work)

    # Wait for all futures
    wait(futures)
    client.close()

In [2]:
#NBVAL_IGNORE_OUTPUT

# Start Dask cluster
cluster = LocalCluster(processes=False, silence_logs=False)

# Generate shot data.
generate_shotdata(param)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: inproc://192.168.0.3/28471/1
distributed.worker - INFO -       Start worker at: inproc://192.168.0.3/28471/2
distributed.worker - INFO -          Listening to:          inproc192.168.0.3
distributed.worker - INFO - Waiting to connect to: inproc://192.168.0.3/28471/1
distributed.worker - INFO - -------------------------------------------------
distributed.worker - INFO -               Threads:                          4
distributed.worker - INFO -                Memory:                    8.59 GB
distributed.worker - INFO -       Local Directory: /Users/ggorman/projects/devito-deadlock/examples/seismic/tutorials/dask-worker-space/worker-xaw2kq2i
distributed.worker - INFO - -------------------------------------------------
distributed.scheduler - INFO - Register inproc://192.168.0.3/28471/2
distributed.worker - INFO -         Registered to: inproc://192.168.0.3/28471/1
distributed.worker - INFO

distributed.worker - DEBUG - Heartbeat skipped: channel busy
distributed.worker - DEBUG - future state: generate_shotdata_i-edcef29f67023b2dd8a158460418afaf - RUNNING
distributed.worker - DEBUG - future state: generate_shotdata_i-d26f9c7789976da55fdbdd4592bdf588 - RUNNING
distributed.worker - DEBUG - future state: generate_shotdata_i-0d9b75bf1cf73b96589039273788acb2 - RUNNING
CustomCompiler: compiled /var/folders/qr/4hc1nw7d3hj55jbd1z7kkrvh0000gn/T/devito-t6mur2ul/0d9992d761763918da7a0cc7fe74d7d3c280a8ce.c [0.71 s]
distributed.worker - DEBUG - future state: generate_shotdata_i-d9f1fd210c2ec1125ec8278575907073 - RUNNING
distributed.worker - DEBUG - Heartbeat skipped: channel busy
distributed.worker - DEBUG - future state: generate_shotdata_i-edcef29f67023b2dd8a158460418afaf - RUNNING
section0<714,181,181> with OI=1.95 computed in 0.213 s [3.52 GFlops/s, 0.11 GPts/s]
distributed.worker - DEBUG - future state: generate_shotdata_i-d26f9c7789976da55fdbdd4592bdf588 - RUNNING
section1<714,1> 

distributed.scheduler - INFO - Remove client Client-d7515778-549b-11e8-af37-3200188f04a0
distributed.client - DEBUG - Release key generate_shotdata_i-edcef29f67023b2dd8a158460418afaf
distributed.worker - DEBUG - Deleted 1 keys
distributed.client - DEBUG - Release key generate_shotdata_i-d9f1fd210c2ec1125ec8278575907073
distributed.client - DEBUG - Release key generate_shotdata_i-904e6c7f49a6b6574d427268d095eda3
distributed.worker - DEBUG - Deleted 1 keys
distributed.client - DEBUG - Release key generate_shotdata_i-d26f9c7789976da55fdbdd4592bdf588
distributed.worker - DEBUG - Deleted 1 keys
distributed.client - DEBUG - Release key generate_shotdata_i-0d9b75bf1cf73b96589039273788acb2
distributed.worker - DEBUG - Deleted 1 keys
distributed.worker - DEBUG - Deleted 1 keys
distributed.scheduler - INFO - Close client connection: Client-d7515778-549b-11e8-af37-3200188f04a0


## Dask specifics

Previously we defined a function to calculate the individual contribution to the functional and gradient for each shot, which was then used in a loop over all shots. However, when using distributed frameworks such as Dask we instead think in terms of creating a worklist which gets *mapped* onto the worker pool. The sum reduction is also performed in parallel. For now however we assume that the scipy.optimize.minimize itself is running on the *master* process; this is a reasonable simplification because the computational cost of calculating (f, g) far exceeds the other compute costs.

Because we want to be able to use standard reduction operators such as sum on (f, g) we first define it as a type so that we can define the `__add__` (and `__rand__` method).

In [3]:
# Define a type to store the functional and gradient.
class fg_pair:
    def __init__(self, f, g):
        self.f = f
        self.g = g
    
    def __add__(self, other):
        f = self.f + other.f
        g = self.g + other.g
        
        return fg_pair(f, g)
    
    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return self.__add__(other)

## Create operators for gradient based inversion
To perform the inversion we are going to use [scipy.optimize.minimize(method=’L-BFGS-B’)](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html#optimize-minimize-lbfgsb).

First we define the functional, ```f```, and gradient, ```g```, operator (i.e. the function ```fun```) for a single shot of data. This is the work that is going to be performed by the worker on a unit of data.

In [4]:
from devito import Function

# Create FWI gradient kernel for a single shot
def fwi_gradient_i(param):
    from devito import clear_cache

    # Need to clear the workers cache.
    clear_cache()

    # Load the current model and the shot data for this worker.
    # Note, unlike the serial example the model is not passed in
    # as an argument. Broadcasting large datasets is considered
    # a programming anti-pattern and at the time of writing it
    # it only worked relaiably with Dask master. Therefore, the
    # the model is communicated via a file.
    model0 = get_current_model(param)
    
    dt = model0.critical_dt

    src, rec = load_shot_data(param['shot_id'], dt)

    # Set up solver.
    solver = AcousticWaveSolver(model0, src, rec, space_order=4)

    # Compute simulated data and full forward wavefield u0
    d, u0, _ = solver.forward(src=src, m=model0.m, save=True)
        
    # Compute the data misfit (residual) and objective function
    residual = Receiver(name='rec', grid=model0.grid,
                        time_range=rec.time_range,
                        coordinates=rec.coordinates.data)

    residual.data[:] = d.data[:] - rec.data[:]
    f = .5*np.linalg.norm(residual.data.flatten())**2
    
    # Compute gradient using the adjoint-state method. Note, this
    # backpropagates the data misfit through the model.
    grad = Function(name="grad", grid=model0.grid)
    solver.gradient(rec=residual, u=u0, m=model0.m, grad=grad)
    
    # Copying here to avoid a (probably overzealous) destructor deleting
    # the gradient before Dask has had a chance to communicate it.
    g = np.array(grad.data[:])
    
    # return the objective functional and gradient.
    return fg_pair(f, g)

Define the global functional-gradient operator. This does the following:
* Maps the worklist (shots) to the workers so that the invidual contributions to (f, g) are computed.
* Sum individual contributions to (f, g) and returns the result.

In [5]:
def fwi_gradient(model, param):
    # Dump a copy of the current model for the workers
    # to pick up when they are ready.
    param['model'] = "model_0.npy"
    dump_model(param, model)

    # Define work list
    work = [dict(param) for i in range(param['nshots'])]
    for i in  range(param['nshots']):
        work[i]['shot_id'] = i
        
    # Distribute worklist to workers.
    client = Client(cluster)
    fgi = client.map(fwi_gradient_i, work, retries=1)
    
    # Perform reduction.
    fg = client.submit(sum, fgi).result()
    
    client.close()
    
    # L-BFGS in scipy expects a flat array in 64-bit floats.
    return fg.f, fg.g.flatten().astype(np.float64)

## FWI with L-BFGS-B
Equipped with a function to calculate the functional and gradient, we are finally ready to define the optimization function.

In [6]:
from scipy import optimize

# Define bounding box constraints on the solution.
def apply_box_constraint(m):
    # Maximum possible 'realistic' velocity is 3.5 km/sec
    # Minimum possible 'realistic' velocity is 2 km/sec
    return np.clip(m, 1/3.5**2, 1/2**2)

# Many optimization methods in scipy.optimize.minimize accept a callback
# function that can operate on the solution after every iteration. Here
# we use this to apply box constraints and to monitor the true relative
# solution error.
relative_error = []
def fwi_callbacks(x):
    # Apply boundary constraint
    x.data[:] = apply_box_constraint(x)
    
    # Calculate true relative error
    true_x = get_true_model().m.data.flatten()
    relative_error.append(np.linalg.norm((x-true_x)/true_x))

def fwi(model, param, ftol=0.1, maxiter=5):
    result = optimize.minimize(fwi_gradient,
                               model.m.data.flatten().astype(np.float64),
                               args=(param, ), method='L-BFGS-B', jac=True,
                               callback=fwi_callbacks,
                               options={'ftol':ftol,
                                        'maxiter':maxiter,
                                        'disp':True})

    return result

We now apply our FWI function and have a look at the result.

In [ ]:
#NBVAL_IGNORE_OUTPUT

model0 = get_initial_model()

# Baby steps
result = fwi(model0, param)

# Print out results of optimizer.
print(result)

distributed.worker - DEBUG - Heartbeat: inproc://192.168.0.3/28471/2
distributed.scheduler - INFO - Receive client connection: Client-e0d8e868-549b-11e8-af37-3200188f04a0
distributed.client - DEBUG - Started scheduling coroutines. Synchronized
distributed.client - DEBUG - map(fwi_gradient_i, ...)
distributed.client - DEBUG - Submit sum(...), sum-6ffcd29f45e14f017a094a410b952cc4
distributed.worker - DEBUG - Execute key: fwi_gradient_i-2ba3e9737236998fc7de4d7bd84c114f worker: inproc://192.168.0.3/28471/2
distributed.worker - DEBUG - Execute key: fwi_gradient_i-0b093c68726fc150e55ecc6ab1fb8079 worker: inproc://192.168.0.3/28471/2
distributed.worker - DEBUG - Execute key: fwi_gradient_i-d3e4e3bd24b5189e027d0c11a14744c5 worker: inproc://192.168.0.3/28471/2
distributed.worker - DEBUG - Execute key: fwi_gradient_i-ca9315951f575b60f2f16a9383c82e02 worker: inproc://192.168.0.3/28471/2
distributed.worker - DEBUG - Heartbeat: inproc://192.168.0.3/28471/2
distributed.worker - DEBUG - Heartbeat: in

distributed.worker - DEBUG - future state: fwi_gradient_i-d3e4e3bd24b5189e027d0c11a14744c5 - RUNNING
distributed.worker - DEBUG - future state: fwi_gradient_i-ca9315951f575b60f2f16a9383c82e02 - RUNNING
distributed.worker - DEBUG - Heartbeat: inproc://192.168.0.3/28471/2
distributed.worker - DEBUG - Heartbeat skipped: channel busy
distributed.worker - DEBUG - future state: fwi_gradient_i-2ba3e9737236998fc7de4d7bd84c114f - RUNNING
distributed.worker - DEBUG - future state: fwi_gradient_i-0b093c68726fc150e55ecc6ab1fb8079 - RUNNING
distributed.worker - DEBUG - future state: fwi_gradient_i-d3e4e3bd24b5189e027d0c11a14744c5 - RUNNING
distributed.worker - DEBUG - future state: fwi_gradient_i-ca9315951f575b60f2f16a9383c82e02 - RUNNING
distributed.worker - DEBUG - Heartbeat: inproc://192.168.0.3/28471/2
distributed.worker - DEBUG - future state: fwi_gradient_i-2ba3e9737236998fc7de4d7bd84c114f - RUNNING
distributed.worker - DEBUG - future state: fwi_gradient_i-0b093c68726fc150e55ecc6ab1fb8079 - R

In [ ]:
#NBVAL_SKIP

# Show what the update does to the model
from examples.seismic import plot_image, plot_velocity

model0.m.data[:] = result.x.astype(np.float32).reshape(model0.m.data.shape)
model0.vp = np.sqrt(1. / model0.m.data[40:-40, 40:-40])
plot_velocity(model0)

In [ ]:
#NBVAL_SKIP

# Plot percentage error
plot_image(100*np.abs(model0.vp-get_true_model().vp.data)/get_true_model().vp.data, vmax=15, cmap="hot")

In [ ]:
#NBVAL_SKIP
import matplotlib.pyplot as plt

# Plot objective function decrease
plt.figure()
plt.loglog(relative_error)
plt.xlabel('Iteration number')
plt.ylabel('True relative error')
plt.title('Convergence')
plt.show()

In [ ]:
# Shutdown cluster
cluster.close()

<sup>This notebook is part of the tutorial "Optimised Symbolic Finite Difference Computation with Devito" presented at the Intel® HPC Developer Conference 2017.</sup>